## Web Scraping Songs & Lyrics

In [1]:
import pymysql
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

In [2]:
# Define an empty dataframe
songs = pd.DataFrame(columns = ['title', 'artist', 'year'])
x = 0
y = 0

# Scrape the Billboard hot 100 yearly chart for 10 years
for i in range(2011, 2021):
    url = "https://www.billboard.com/charts/year-end/{}/hot-100-songs".format(i)
    header = {'User-Agent' : 'Mozilla/5.0'}
    page = requests.get(url, headers = header)
    doc = BeautifulSoup(page.content, 'html.parser')
    div1 = doc.find_all('div', class_ = 'ye-chart-item__title')
    div2 = doc.find_all('div', class_ = 'ye-chart-item__artist')
    for a in div1:
        songs.loc[x, 'title'] = a.text
        x = x + 1
    for b in div2:
        songs.loc[y, 'artist'] = b.text
        songs.loc[y, 'year'] = i
        y = y + 1
    print('Scraping year', i)

songs

Scraping year 2011
Scraping year 2012
Scraping year 2013
Scraping year 2014
Scraping year 2015
Scraping year 2016
Scraping year 2017
Scraping year 2018
Scraping year 2019
Scraping year 2020


,title,artist,year
0,\nRolling In The Deep\n,\n\nAdele\n\n,2011
1,\nParty Rock Anthem\n,\nLMFAO Featuring Lauren Bennett & GoonRock\n,2011
2,\nFirework\n,\n\nKaty Perry\n\n,2011
3,\nE.T.\n,\nKaty Perry Featuring Kanye West\n,2011
4,\nGive Me Everything\n,"\nPitbull Featuring Ne-Yo, Afrojack & Nayer\n",2011
...,...,...,...
993,\nMore Than My Hometown\n,\n\nMorgan Wallen\n\n,2020
994,\nLovin' On You\n,\n\nLuke Combs\n\n,2020
995,\nSaid Sum\n,\n\nMoneybagg Yo\n\n,2020
996,\nSlide\n,\nH.E.R. Featuring YG\n,2020


In [3]:
# Strip the line break
songs['title'] = songs['title'].replace('\n','', regex=True)
songs['artist'] = songs['artist'].replace('\n','', regex=True)
songs

,title,artist,year
0,Rolling In The Deep,Adele,2011
1,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock,2011
2,Firework,Katy Perry,2011
3,E.T.,Katy Perry Featuring Kanye West,2011
4,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",2011
...,...,...,...
993,More Than My Hometown,Morgan Wallen,2020
994,Lovin' On You,Luke Combs,2020
995,Said Sum,Moneybagg Yo,2020
996,Slide,H.E.R. Featuring YG,2020


In [4]:
# Strip the song title and artists for lyrics searching
songs_strip = songs.copy()
#songs_strip['title'] = songs_strip['title'].replace('&', 'and', regex=True)
#songs_strip['artist'] = songs_strip['artist'].replace('&', 'and', regex=True)
songs_strip['title'] = songs_strip['title'].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
songs_strip['artist'] = songs_strip['artist'].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
songs_strip['artist'] = songs_strip['artist'].replace(' Featuring.+', '', regex=True)
songs_strip['title'] = songs_strip['title'].replace(' ', '-', regex=True)
songs_strip['artist'] = songs_strip['artist'].replace(' ', '-', regex=True)
songs_strip['concat'] = songs_strip['artist'] + '-' + songs_strip['title']
songs_strip

,title,artist,year,concat
0,Rolling-In-The-Deep,Adele,2011,Adele-Rolling-In-The-Deep
1,Party-Rock-Anthem,LMFAO,2011,LMFAO-Party-Rock-Anthem
2,Firework,Katy-Perry,2011,Katy-Perry-Firework
3,ET,Katy-Perry,2011,Katy-Perry-ET
4,Give-Me-Everything,Pitbull,2011,Pitbull-Give-Me-Everything
...,...,...,...,...
993,More-Than-My-Hometown,Morgan-Wallen,2020,Morgan-Wallen-More-Than-My-Hometown
994,Lovin-On-You,Luke-Combs,2020,Luke-Combs-Lovin-On-You
995,Said-Sum,Moneybagg-Yo,2020,Moneybagg-Yo-Said-Sum
996,Slide,HER,2020,HER-Slide


In [5]:
# Scraping lyrics
c = 0

for index, row in songs_strip.iterrows():
    concat = str(row['concat'])
    try:
        url_lyrics = "https://genius.com/" + concat + "-lyrics"
        page_lyrics = requests.get(url_lyrics, headers = header)
        doc_lyrics = BeautifulSoup(page_lyrics.content, 'html.parser')
        div = doc_lyrics.find('div', class_ = 'lyrics').get_text()
        div = re.sub(r'[\(\[].*?[\)\]]', '', div)
        songs_strip.loc[c, 'lyrics'] = div
    except:
        songs_strip.loc[c, 'lyrics'] = None
    c = c + 1

songs_strip

,title,artist,year,concat,lyrics
0,Rolling-In-The-Deep,Adele,2011,Adele-Rolling-In-The-Deep,\n\n\nThere's a fire starting in my heart\nRea...
1,Party-Rock-Anthem,LMFAO,2011,LMFAO-Party-Rock-Anthem,\n\n\nParty Rock\nYeah\nWoo!\nLet's go!\n\n\nP...
2,Firework,Katy-Perry,2011,Katy-Perry-Firework,\n\n\nDo you ever feel like a plastic bag\nDri...
3,ET,Katy-Perry,2011,Katy-Perry-ET,\n\n\nYou're so hypnotizing\nCould you be the ...
4,Give-Me-Everything,Pitbull,2011,Pitbull-Give-Me-Everything,\n\n\nMe not workin' hard? Yeah right\nPicture...
...,...,...,...,...,...
993,More-Than-My-Hometown,Morgan-Wallen,2020,Morgan-Wallen-More-Than-My-Hometown,"\n\n\nGirl, our mamas are best friends and so ..."
994,Lovin-On-You,Luke-Combs,2020,Luke-Combs-Lovin-On-You,\n\n\nDon't get me wrong\nI like a bobber on t...
995,Said-Sum,Moneybagg-Yo,2020,Moneybagg-Yo-Said-Sum,"\n\n\n\n\n\nHuh? \nAh, I thought a broke nigga..."
996,Slide,HER,2020,HER-Slide,\n\n\nYou always wearin' them glasses\nYou don...


# Spotify ID

In [6]:
import base64
import json

In [7]:
# updating syntax in dataframe

# titles
songs_strip["title"][10] = 'S-&-M'
songs_strip["title"][17] = 'Perfect'
songs_strip["title"][138] = 'Ni**as-in-Paris'
songs_strip["title"][239] = 'F**kin-Problems'
songs_strip["title"][285] = 'U.O.E.N.O.'
songs_strip["title"][445] = 'I-Dont-F**k-With-You'
songs_strip["title"][459] = '7/11'
songs_strip["title"][547] = 'H.O.L.Y.'
songs_strip["title"][628] = '1-800-273-8255'
songs_strip["title"][649] = 'T-Shirt'
songs_strip["title"][781] = '1-800-273-8255'
songs_strip["title"][825] = 'Ransom'
songs_strip["title"][991] = 'P*$$y-Fairy-OTW'
songs_strip["title"][996] = 'H.E.R.'


# artists
for row in songs_strip.loc[songs_strip["artist"] == "Pnk"].iterrows():
    i = row[0]
    songs_strip["artist"][i] = "Pink"

for row in songs_strip.loc[songs_strip["artist"] == "Keha"].iterrows():
    i = row[0]
    songs_strip["artist"][i] = 'Ke$ha'

for row in songs_strip.loc[songs_strip["artist"] == "FarEast-Movement"].iterrows():
    i = row[0]
    songs_strip["artist"][i] = 'Far-East-Movement'

songs_strip["artist"][80] = 'Brad-Paisley'
songs_strip["artist"][136] = 'Owl-City'

for row in songs_strip.loc[songs_strip["artist"] == "NeYo"].iterrows():
    i = row[0]
    songs_strip["artist"][i] = 'Ne-Yo'

for row in songs_strip.loc[songs_strip["artist"] == "OT-Genasis"].iterrows():
    i = row[0]
    songs_strip["artist"][i] = 'O.T.-Genasis'

songs_strip["artist"][183] = 'Ke$ha'
songs_strip["artist"][184] = 'T-Pain'
songs_strip["artist"][221] = 'will.i.am,-Britney-Spears'
songs_strip["artist"][239] = 'A$AP-Rocky'
songs_strip["artist"][243] = 'Lana-Del-Rey'
songs_strip["artist"][293] = 'will.i.am'
songs_strip["artist"][385] = 'T.I.'
songs_strip["artist"][388] = 'Mike-Will-Made-It'
songs_strip["artist"][448] = 'T-Wayne'
songs_strip["artist"][517] = 'G-Eazy'
songs_strip["artist"][531] = 'Pink'
songs_strip["artist"][624] = 'Kygo'
songs_strip["artist"][638] = 'Machine-Gun-Kelly'
songs_strip["artist"][723] = 'Marshmello'
songs_strip["artist"][727] = 'G-Eazy'
songs_strip["artist"][742] = 'G-Eazy'
songs_strip["artist"][757] = 'Selena-Gomez'
songs_strip["artist"][771] = 'A$AP-Ferg'
songs_strip["artist"][778] = 'Nio-Garcia'
songs_strip["artist"][780] = 'N.E.R.D.'
songs_strip["artist"][799] = 'Post-Malone'
songs_strip["artist"][848] = 'Gucci-Mane'
songs_strip["artist"][855] = 'Ellie-Goulding'
songs_strip["artist"][920] = 'Dan+Shay'
songs_strip["artist"][932] = 'Jawsh-685'
songs_strip["artist"][947] = 'Black-Eyed-Peas'
songs_strip["artist"][951] = 'Juice-WRLD'
songs_strip["artist"][985] = 'Kane-Brown'



In [8]:
songs_strip.loc[songs_strip["artist"] == "AAP-Ferg"]

,title,artist,year,concat,lyrics


In [11]:
# Client Credentials Flow

url = 'https://accounts.spotify.com/api/token'
headers = {}
data = {}

# enter your credentials from spotify developer dashboard
client_id = '6f8b3ac1241549f3bed8987b4df9e142'
client_secret = '512f0b3b62984a5d836a7c09ec82052b'

message = f"{client_id}:{client_secret}"
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

headers['Authorization'] = f"Basic {base64_message}"
data['grant_type'] = 'client_credentials'

r = requests.post(url, headers = headers, data=data)

print(json.dumps(r.json(), indent = 2))

token = r.json()['access_token']

{
  "access_token": "BQD8v457onEE2OA1ao4Z00ZsvwuvT40Rs3dNnNovg_eNu8_ZS2meJVkmTRN9tA3hvPVt9gNOv_Z_P5H8SoM",
  "token_type": "Bearer",
  "expires_in": 3600
}


In [12]:
# Search API

base_url = 'https://api.spotify.com/v1/search'
spotify_id = [] 
headers = { "Authorization": "Bearer " + token }
count = 1
for i in range(0, len(songs_strip)):
    song = re.sub('-', '%20', songs_strip["title"][i])
    artist = re.sub('-', '%20', songs_strip["artist"][i])

    query = f"track:{song}%20artist:{artist}&type=track&limit=1"

    search_url = base_url + "?q=" + query


    #query = "track:Firework%20artist:Katy%20Perry&type=track&limit=1"

    #search_url = base_url + "?q=" + query

    res = requests.get(search_url, headers=headers)

    doc = res.json()

    #print(json.dumps(doc, indent= 4))

    tracks = doc['tracks']

    all_items = tracks['items']
    if all_items == []:
        spotify_id.append("error")
    else:
        first_result = all_items[0]

        track_id = first_result["id"]

        spotify_id.append(track_id)
    print("Scraped spotify_id " + str(count) + "/" + str(len(songs_strip)))
    
    count += 1



Scraped spotify_id 1/998
Scraped spotify_id 2/998
Scraped spotify_id 3/998
Scraped spotify_id 4/998
Scraped spotify_id 5/998
Scraped spotify_id 6/998
Scraped spotify_id 7/998
Scraped spotify_id 8/998
Scraped spotify_id 9/998
Scraped spotify_id 10/998
Scraped spotify_id 11/998
Scraped spotify_id 12/998
Scraped spotify_id 13/998
Scraped spotify_id 14/998
Scraped spotify_id 15/998
Scraped spotify_id 16/998
Scraped spotify_id 17/998
Scraped spotify_id 18/998
Scraped spotify_id 19/998
Scraped spotify_id 20/998
Scraped spotify_id 21/998
Scraped spotify_id 22/998
Scraped spotify_id 23/998
Scraped spotify_id 24/998
Scraped spotify_id 25/998
Scraped spotify_id 26/998
Scraped spotify_id 27/998
Scraped spotify_id 28/998
Scraped spotify_id 29/998
Scraped spotify_id 30/998
Scraped spotify_id 31/998
Scraped spotify_id 32/998
Scraped spotify_id 33/998
Scraped spotify_id 34/998
Scraped spotify_id 35/998
Scraped spotify_id 36/998
Scraped spotify_id 37/998
Scraped spotify_id 38/998
Scraped spotify_id 39

In [13]:
# can then look up index of error to see what the problem/typo is
spotify_id.count("error")

7

In [14]:
# verify spotify id by searching for the song
# url = 'https://open.spotify.com/track/4cSSL3YafYjM3yjgFO1vJg' + spotify_id

In [14]:
songs_strip["spotify_id"] = spotify_id

# Audio Features

In [15]:
# Getting audio features
import pprint
af_url_base = 'https://api.spotify.com/v1/audio-features/'
audio_features = []
count_af = 1
for i in spotify_id:
    af_url = af_url_base + i
    headers = { "Authorization": "Bearer " + token,
              "Accept": "application/json",
              "Content-Type": "application/json"}
    try:
        af_req = requests.get(af_url, headers=headers)
        af_req.raise_for_status()
        afdoc = af_req.json()
        audio_features.append(afdoc)
    except requests.exceptions.RequestException as e:
        audio_features.append("Null")
        print(e)
    print("Getting Audio Features for id " + str(count_af) + "/" + str(len(songs_strip)))
    count_af += 1


Getting Audio Features for id 1/998
Getting Audio Features for id 2/998
Getting Audio Features for id 3/998
Getting Audio Features for id 4/998
Getting Audio Features for id 5/998
Getting Audio Features for id 6/998
Getting Audio Features for id 7/998
Getting Audio Features for id 8/998
Getting Audio Features for id 9/998
Getting Audio Features for id 10/998
Getting Audio Features for id 11/998
Getting Audio Features for id 12/998
Getting Audio Features for id 13/998
Getting Audio Features for id 14/998
Getting Audio Features for id 15/998
Getting Audio Features for id 16/998
Getting Audio Features for id 17/998
Getting Audio Features for id 18/998
Getting Audio Features for id 19/998
Getting Audio Features for id 20/998
Getting Audio Features for id 21/998
Getting Audio Features for id 22/998
Getting Audio Features for id 23/998
Getting Audio Features for id 24/998
Getting Audio Features for id 25/998
Getting Audio Features for id 26/998
Getting Audio Features for id 27/998
Getting Au

In [16]:
# Seperating features
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
Type = []
uri = []
track_href = []
duration_ms = []
time_signature = []

In [17]:
for i in audio_features:
    if i != 'Null':
        danceability.append(i['danceability'])
        energy.append(i['energy'])
        key.append(i['key'])
        loudness.append(i['loudness'])
        mode.append(i['mode'])
        speechiness.append(i['speechiness'])
        acousticness.append(i['acousticness'])
        instrumentalness.append(i['instrumentalness'])
        liveness.append(i['liveness'])
        valence.append(i['valence'])
        tempo.append(i['tempo'])
        Type.append(i['type'])
        uri.append(i['uri'])
        track_href.append(i['track_href'])
        duration_ms.append(i['duration_ms'])
        time_signature.append(i['time_signature'])
    else:
        danceability.append('Null')
        energy.append('Null')
        key.append('Null')
        loudness.append('Null')
        mode.append('Null')
        speechiness.append('Null')
        acousticness.append('Null')
        instrumentalness.append('Null')
        liveness.append('Null')
        valence.append('Null')
        tempo.append('Null')
        Type.append('Null')
        uri.append('Null')
        track_href.append('Null')
        duration_ms.append('Null')
        time_signature.append('Null')

In [18]:
songs_strip["danceability"] = danceability
songs_strip["energy"] = energy
songs_strip["key"] = key
songs_strip["loudness"] = loudness
songs_strip["mode"] = mode
songs_strip["speechiness"] = speechiness
songs_strip["acousticness"] = acousticness
songs_strip["instrumentalness"] = instrumentalness
songs_strip["liveness"] = liveness
songs_strip["valence"] = valence
songs_strip["tempo"] = tempo
songs_strip["Type"] = Type
songs_strip["uri"] = uri
songs_strip["track_href"] = track_href
songs_strip["duration_ms"] = duration_ms
songs_strip["time_signature"] = time_signature
songs_strip

,title,artist,year,concat,lyrics,spotify_id,danceability,energy,key,loudness,...,acousticness,instrumentalness,liveness,valence,tempo,Type,uri,track_href,duration_ms,time_signature
0,Rolling-In-The-Deep,Adele,2011,Adele-Rolling-In-The-Deep,\n\n\nThere's a fire starting in my heart\nRea...,1c8gk2PeTE04A1pIDH9YMk,0.73,0.769,8,-5.114,...,0.138,0,0.0473,0.507,104.948,audio_features,spotify:track:1c8gk2PeTE04A1pIDH9YMk,https://api.spotify.com/v1/tracks/1c8gk2PeTE04...,228093,4
1,Party-Rock-Anthem,LMFAO,2011,LMFAO-Party-Rock-Anthem,\n\n\nParty Rock\nYeah\nWoo!\nLet's go!\n\n\nP...,0IkKz2J93C94Ei4BvDop7P,0.75,0.727,5,-4.21,...,0.0189,0,0.266,0.359,129.993,audio_features,spotify:track:0IkKz2J93C94Ei4BvDop7P,https://api.spotify.com/v1/tracks/0IkKz2J93C94...,262173,4
2,Firework,Katy-Perry,2011,Katy-Perry-Firework,\n\n\nDo you ever feel like a plastic bag\nDri...,4lCv7b86sLynZbXhfScfm2,0.638,0.826,8,-4.968,...,0.139,0,0.0803,0.649,124.072,audio_features,spotify:track:4lCv7b86sLynZbXhfScfm2,https://api.spotify.com/v1/tracks/4lCv7b86sLyn...,227880,4
3,ET,Katy-Perry,2011,Katy-Perry-ET,\n\n\nYou're so hypnotizing\nCould you be the ...,5v0OGuLQbc3kLWS7fHklzU,0.53,0.902,5,-6.105,...,0.00595,0.102,0.139,0.602,150.015,audio_features,spotify:track:5v0OGuLQbc3kLWS7fHklzU,https://api.spotify.com/v1/tracks/5v0OGuLQbc3k...,589734,4
4,Give-Me-Everything,Pitbull,2011,Pitbull-Give-Me-Everything,\n\n\nMe not workin' hard? Yeah right\nPicture...,4QNpBfC0zvjKqPJcyqBy9W,0.671,0.939,8,-3.206,...,0.191,0,0.298,0.53,129.024,audio_features,spotify:track:4QNpBfC0zvjKqPJcyqBy9W,https://api.spotify.com/v1/tracks/4QNpBfC0zvjK...,252307,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,More-Than-My-Hometown,Morgan-Wallen,2020,Morgan-Wallen-More-Than-My-Hometown,"\n\n\nGirl, our mamas are best friends and so ...",5OELUCYgOHKFAvCERnAvfS,0.62,0.869,6,-5.479,...,0.615,0,0.131,0.597,126.019,audio_features,spotify:track:5OELUCYgOHKFAvCERnAvfS,https://api.spotify.com/v1/tracks/5OELUCYgOHKF...,216573,4
994,Lovin-On-You,Luke-Combs,2020,Luke-Combs-Lovin-On-You,\n\n\nDon't get me wrong\nI like a bobber on t...,0nYvjcSlCgjcwogQAwIwNp,0.572,0.949,4,-4.865,...,0.00165,0.000195,0.163,0.53,118.974,audio_features,spotify:track:0nYvjcSlCgjcwogQAwIwNp,https://api.spotify.com/v1/tracks/0nYvjcSlCgjc...,194867,4
995,Said-Sum,Moneybagg-Yo,2020,Moneybagg-Yo-Said-Sum,"\n\n\n\n\n\nHuh? \nAh, I thought a broke nigga...",3sKz6Sd72K0ofPWcJPPk6H,0.929,0.667,8,-6.789,...,0.0185,0,0.1,0.274,126.998,audio_features,spotify:track:3sKz6Sd72K0ofPWcJPPk6H,https://api.spotify.com/v1/tracks/3sKz6Sd72K0o...,155168,4
996,H.E.R.,HER,2020,HER-Slide,\n\n\nYou always wearin' them glasses\nYou don...,0KeE5dKNErVA9ZWx2VfjOC,0.627,0.502,11,-12.351,...,0.181,0,0.129,0.356,110.255,audio_features,spotify:track:0KeE5dKNErVA9ZWx2VfjOC,https://api.spotify.com/v1/tracks/0KeE5dKNErVA...,176790,4


## Database

In [19]:
# Create MySQL database
SQL_DB = "hits"
SQL_TABLE = "hits_lyrics"
SQL_TABLE_DEF = "(" + \
            "id INT NOT NULL AUTO_INCREMENT PRIMARY KEY" + \
            ",title VARCHAR(100)" + \
            ",artist VARCHAR(100)" + \
            ",year YEAR(4)" + \
            ",year TEXT" + \
            ")"
conn = mysql.connector.connect(host='localhost', user='root')
cursor = conn.cursor()
query1 = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
print(query1)
cursor.execute(query1);
        
query2 = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE + " " + SQL_TABLE_DEF + ";";
print(query2)
cursor.execute(query2);
cursor.close()
conn.close()

# Back up codes

In [113]:
# In case we have to fix the songs with errors:

#for idx, i in  enumerate(audio_features):
    if i == "Null":
        af_url = af_url_base + i
        headers = { "Authorization": "Bearer " + token,
                  "Accept": "application/json",
                  "Content-Type": "application/json"}
        try:
            af_req = requests.get(af_url, headers=headers)
            af_req.raise_for_status()
            afdoc = af_req.json()
            audio_features[index] = afdoc
        except requests.exceptions.RequestException as e:
            audio_features[index] = "Null"
            print(e)
# check if still null:

pprint.pprint(audio_features[92])

400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spotify.com/v1/audio-features/Null
400 Client Error: Bad Request for url: https://api.spo

In [117]:
# Counting how many nulls
nnn = 0
for i in audio_features:
    if i == "Null":
        nnn += 1
nnn

13